In [5]:
# import needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
# create a function to scrape player performance to evaluate streakyness
def scrape_NBA_player_data():
    

    df = pd.DataFrame(columns = ["Player","Date","Age","Team"," ","Opp","Result","MP","FG","FGA","FG%","2P",
    "2PA","2P%","3P","3PA","3P%","FT","FTA","FT%","TS%","PTS","Pos."])
   
       

    

    # loop through each page, multiple batches needed so it doesnt time out 
    for x in range(24000,26000,200): 

    # log in to Stathead using cookies
        cookies = {
            'srcssfull': 'yes',
            'csrf_token': '422d838f3ce7e614378cac600f819366',
            'stathead_user': 'Cashley%3A%3ACashley%3A%3A066c5623fa026f571b9402e6958a21b4%3A%3A55797622',
            'stathead_site': 'bbr',
            'stathead_type': 'trial',
        }

        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            # Requests sorts cookies= alphabetically
            # 'Cookie': 'srcssfull=yes; csrf_token=422d838f3ce7e614378cac600f819366; stathead_user=Cashley%3A%3ACashley%3A%3A066c5623fa026f571b9402e6958a21b4%3A%3A55797622; stathead_site=bbr; stathead_type=trial',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
            'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
        }
        
        url = f"https://stathead.com/basketball/player-game-finder.cgi?request=1&game_num_max=82&comp_type=reg&order_by=name_display_csk&match=player_game&season_start=1&year_max=2022&season_end=-1&order_by_asc=0&game_num_type=season&game_num_min=1&comp_id=NBA&year_min=2022&cstat\\[1\\]=fga&ccomp\\[1\\]=gt&cval\\[1\\]=5&offset={x}"
        
        response = requests.get(url, cookies=cookies, headers=headers)
        #print(response.content)
        
        # create beautiful soup object from HTML
        soup = BeautifulSoup(response.content, features="lxml")
        
        # use getText()to extract the headers into a list
        titles = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
        
        # first, find only column headers
        headers = titles[1:titles.index("Pos.")+1]
        
        # then, exclude first set of column headers (duplicated)
        titles = titles[titles.index("Pos.")+1:]
        
        # next, grab all data from rows (avoid first row)
        rows = soup.findAll('tr')[1:]
        player_stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
    
        #print(player_stats, end='\n')
    
    
            # remove empty elements
        player_stats = [e for e in player_stats if e != []]
        
        # create a dataframe with all aquired info
        players = pd.DataFrame(player_stats, columns = headers)

        df = df.reset_index(drop=True)
        
        # append new dataframe to df
        df = df.append(players)
        
   

        
        

        
    # print df
    print(df.info)
    print(df.head())
    # export to csv
    df.to_csv("nba_player_datainsane2.csv", index=False)
    

scrape_NBA_player_data()



<bound method DataFrame.info of                Player        Date     Age Team       Opp     Result  MP FG  \
0       Darius Bazley  2022-02-25  21-258  OKC  NaN  IND  W 129-125  34  6   
1       Darius Bazley  2022-02-28  21-261  OKC  NaN  SAC  L 110-131  29  3   
2       Darius Bazley  2022-03-02  21-263  OKC  NaN  DEN  W 119-107  33  4   
3       Darius Bazley  2022-03-04  21-265  OKC  NaN  MIN  L 101-138  18  1   
4       Darius Bazley  2022-03-06  21-267  OKC  NaN  UTA  L 103-116  36  5   
..                ...         ...     ...  ...  ...  ...        ...  .. ..   
195  Precious Achiuwa  2021-11-11  22-053  TOR  NaN  PHI  W 115-109  30  1   
196  Precious Achiuwa  2021-11-19  22-061  TOR  NaN  SAC   W 108-89  25  3   
197  Precious Achiuwa  2021-11-21  22-063  TOR  NaN  GSW  L 104-119  26  5   
198  Precious Achiuwa  2021-11-24  22-066  TOR  NaN  MEM  W 126-113  28  7   
199  Precious Achiuwa  2021-11-26  22-068  TOR  NaN  IND   L 97-114  33  4   

    FGA  ... ORB DRB TRB AST ST

In [ ]:
#Data cleaned in excel
    #Used filters to get rid of low data players
    #Preformed various calculations to create data which ultimately 
    #was used in finding the standard deviations